In [2]:
import pandas as pd
import copy
import torch
from datasets import load_dataset, Dataset
from dotenv import load_dotenv
# Use a pipeline as a high-level helper
from transformers import pipeline
from tqdm.notebook import tqdm

load_dotenv()
pipe = pipeline("token-classification", model="obi/deid_bert_i2b2")

Device set to use mps:0


In [5]:
# df = pd.read_csv('healthcare_dataset.csv')

In [3]:
dataset = load_dataset("mks-logic/SPY", trust_remote_code=True, faker_random_seed=42)

In [7]:
# dataset

In [4]:
for x in dataset['medical_consultations']:
    # print(x)
    break
print(dataset.column_names)
lst_of_ent_tags = dataset['medical_consultations']['ent_tags']
lst_of_tokens = dataset['medical_consultations']['tokens']
lst_of_trailing_whitespaces = dataset['medical_consultations']['trailing_whitespaces']
lst_of_labels = dataset['medical_consultations']['labels']
print(type(lst_of_ent_tags[0][2]))
print(type(lst_of_tokens[0][2]))
print(type(lst_of_trailing_whitespaces[0][2]))
print(type(lst_of_labels[0][2]))

{'legal_questions': ['tokens', 'trailing_whitespaces', 'labels', 'ent_tags'], 'medical_consultations': ['tokens', 'trailing_whitespaces', 'labels', 'ent_tags']}
<class 'str'>
<class 'str'>
<class 'bool'>
<class 'int'>


In [115]:
print(lst_of_tokens[0])
print(lst_of_tokens[0].index("Christopher"))
print(lst_of_trailing_whitespaces[0])
print(lst_of_labels[0])

['Text', ': ', 'Hi', ', ', 'I ', 'am ', 'Christopher', 'Murillo', ', ', '\n', 'I ', 'am ', 'experiencing ', 'a ', 'sharp ', 'pain ', 'in ', 'the ', 'lower ', 'right ', 'abdomen', ', ', 'occasionally ', 'radiating ', 'to ', 'my ', 'back', '. ', 'This ', 'symptom ', 'has ', 'been ', 'persistent ', 'for ', 'the ', 'past ', '4 ', 'days ', 'now', '. ', 'The ', 'pain ', 'is ', 'worsening ', 'over ', 'time', ', ', 'especially ', 'after ', 'eating ', 'or ', 'engaging ', 'in ', 'physical ', 'activities', '. ', 'I ', 'have ', 'noticed ', 'minor ', 'nausea', ', ', 'but ', 'no ', 'vomiting ', 'or ', 'bleeding', '. ', 'The ', 'pain ', 'is ', 'non', '-', 'specific', ', ', 'meaning ', 'it ', 'does', "n't ", 'seem ', 'to ', 'be ', 'triggered ', 'by ', 'a ', 'specific ', 'food ', 'intake ', 'or ', 'any ', 'other ', 'activity', '. ', '\n\n', 'You ', 'can ', 'reach ', 'me ', 'at ', 'alvarezkenneth@gmail.com', 'or ', '493-290-9635', 'for ', 'any ', 'clarification ', 'or ', 'follow', '-', 'up ', 'questions

In [5]:
from typing import List

def reconstruct_text_from_tokens(tokens: List[str], has_trailing_space: List[bool], labels: List[int]) -> str:
    """
    Reconstructs a text string from a list of tokens and a list of trailing whitespace indicators.

    Args:
        tokens (List[str]): List of token strings.
        has_trailing_space (List[bool]): List of booleans where True means a space should follow the token.

    Returns:
        str: The reconstructed string with appropriate single-space separations.
    
    Raises:
        ValueError: If the lengths of tokens and has_trailing_space do not match.
    """
    if len(tokens) != len(has_trailing_space):
        raise ValueError("Length of tokens and has_trailing_space must be equal.")
    
    pieces = []
    for token, has_space, label in zip(tokens, has_trailing_space, labels):
        pieces.append(token)
        if (label != 14):
            pieces.append(" ")

    return "".join(pieces)


In [6]:
t = reconstruct_text_from_tokens(lst_of_tokens[0],lst_of_trailing_whitespaces[0],lst_of_labels[0])
print(t)

Text: Hi, I am Christopher Murillo , 
I am experiencing a sharp pain in the lower right abdomen, occasionally radiating to my back. This symptom has been persistent for the past 4 days now. The pain is worsening over time, especially after eating or engaging in physical activities. I have noticed minor nausea, but no vomiting or bleeding. The pain is non-specific, meaning it doesn't seem to be triggered by a specific food intake or any other activity. 

You can reach me at alvarezkenneth@gmail.com or 493-290-9635 for any clarification or follow-up questions.

I have a history of appendicitis which was surgically removed when I was 17. I have also had several episodes of stomach ulcers in the past, for which I have taken antacids and antibiotics as prescribed by my previous doctor, Dr. Emily Patel, whose contact information is available at https://facebook.com/healthcareprofessionals. 

Currently, I am not taking any medications, but I have an upcoming doctor's appointment with Dr. Soph

In [7]:
lst_of_docs = [" ".join(lst)
    for lst in lst_of_tokens
]
lst_of_docs = [" ".join(doc.split())
              for doc in lst_of_docs]
print(lst_of_docs[1])

Hello ! I 'm Jacqueline Adams , reaching out for an online consultation due to a recurring issue with tooth sensitivity . It 's been going on for the past 6 months , and it 's been getting gradually worse . Sometimes it 's just a mild tingling sensation , but other times it 's a sharp pain . I 've noticed it 's triggered by eating or drinking something hot or cold . I 've had a few dental fillings in the past , but no major dental work . You can reach me back at christopher17@gmail.com if there 's any additional information required . I 've tried using toothpaste for sensitive teeth , but it does n't seem to be making a difference . My dentist prescribed me a fluoride rinse , but I have n't seen much improvement . In my free time , I usually play online games under the username millerchristina . I 'd appreciate it if you could recommend any products or services that are available at 6771 Johns Shores or can be shipped to my location . Could you please advise on what might be causing th

In [8]:
print(dataset['medical_consultations']['tokens'][0])

['Text', ': ', 'Hi', ', ', 'I ', 'am ', 'Christopher', 'Murillo', ', ', '\n', 'I ', 'am ', 'experiencing ', 'a ', 'sharp ', 'pain ', 'in ', 'the ', 'lower ', 'right ', 'abdomen', ', ', 'occasionally ', 'radiating ', 'to ', 'my ', 'back', '. ', 'This ', 'symptom ', 'has ', 'been ', 'persistent ', 'for ', 'the ', 'past ', '4 ', 'days ', 'now', '. ', 'The ', 'pain ', 'is ', 'worsening ', 'over ', 'time', ', ', 'especially ', 'after ', 'eating ', 'or ', 'engaging ', 'in ', 'physical ', 'activities', '. ', 'I ', 'have ', 'noticed ', 'minor ', 'nausea', ', ', 'but ', 'no ', 'vomiting ', 'or ', 'bleeding', '. ', 'The ', 'pain ', 'is ', 'non', '-', 'specific', ', ', 'meaning ', 'it ', 'does', "n't ", 'seem ', 'to ', 'be ', 'triggered ', 'by ', 'a ', 'specific ', 'food ', 'intake ', 'or ', 'any ', 'other ', 'activity', '. ', '\n\n', 'You ', 'can ', 'reach ', 'me ', 'at ', 'alvarezkenneth@gmail.com', 'or ', '493-290-9635', 'for ', 'any ', 'clarification ', 'or ', 'follow', '-', 'up ', 'questions

In [9]:
df = dataset['medical_consultations'].to_pandas()

In [10]:
df['text'] = df['tokens'].apply(lambda x: "".join(x))
dataset_txt=Dataset.from_pandas(df)

In [18]:
#dataset_txt['text']

In [19]:
# Use a pipeline as a high-level helper
#from transformers import pipeline

#pipe = pipeline("token-classification", model="obi/deid_roberta_i2b2")

In [20]:
#results= pipe{dataset['medical_consultations']}

In [21]:
#dataset['medical_consultations']

In [22]:
#results=[]
#for out in pipe(dataset['medical_consultations']):
    
    #results.append(out)

In [24]:
#dataset_v2 = dataset_txt['text'].map(
          #  pipe,
           # batched=True,
            #batch_size=4,
        # )


In [25]:
# pipe(['my name is president Barack Obama'])

In [26]:
# results=[]
# #with torch.inference_mode():
# for inp in dataset_txt['text']:
#     # print(inp)
#     assert isinstance(inp, str), "not a str"
#     out=pipe([inp], truncation=True)
#     results.append(out)

In [27]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("obi/deid_roberta_i2b2")
model = AutoModelForTokenClassification.from_pretrained("obi/deid_roberta_i2b2")

In [29]:
# inputs = tokenizer(inp, truncation=True, return_tensors='pt')

In [30]:
# outputs = model(**inputs)

In [31]:
# outputs.logits
# tokenizer.decode(outputs)

In [ ]:
from transformers import AutoTokenizer

tokenizer = pipe.tokenizer  # get tokenizer from your existing pipeline

results = []
for inp in tqdm(lst_of_docs):
    assert isinstance(inp, str), "not a str"
    
    # Truncate the input at token level
    inputs = tokenizer(
        inp,
        truncation=True,
        max_length=512,
        return_tensors='pt',
        return_attention_mask=True
    )

    # Decode back to string after truncation
    truncated_text = tokenizer.decode(inputs["input_ids"][0], skip_special_tokens=True)

    # Pass the truncated string to the pipeline
    out = pipe(truncated_text)
    results.append(out)


  0%|          | 0/4491 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [139]:
print(results[1])

[{'entity': 'B-PATIENT', 'score': np.float32(0.99691075), 'index': 6, 'word': 'Jacqueline', 'start': 13, 'end': 23}, {'entity': 'L-PATIENT', 'score': np.float32(0.9775359), 'index': 7, 'word': 'Adams', 'start': 24, 'end': 29}, {'entity': 'U-STAFF', 'score': np.float32(0.7998271), 'index': 107, 'word': 'ch', 'start': 465, 'end': 467}, {'entity': 'U-PATIENT', 'score': np.float32(0.4773383), 'index': 187, 'word': 'mill', 'start': 790, 'end': 794}, {'entity': 'B-LOC', 'score': np.float32(0.9802167), 'index': 210, 'word': '67', 'start': 897, 'end': 899}, {'entity': 'B-LOC', 'score': np.float32(0.6470808), 'index': 211, 'word': '##7', 'start': 899, 'end': 900}, {'entity': 'I-LOC', 'score': np.float32(0.5506426), 'index': 212, 'word': '##1', 'start': 900, 'end': 901}, {'entity': 'I-LOC', 'score': np.float32(0.986521), 'index': 213, 'word': 'Johns', 'start': 902, 'end': 907}, {'entity': 'L-LOC', 'score': np.float32(0.7195891), 'index': 214, 'word': 'Shore', 'start': 908, 'end': 913}, {'entity'

In [40]:
def get_true_entities(tokens, ner_tags):
    """
    this function does something

    Args:
        tokens (List[str]): labeled tokens
        ner_tags (List[str]) Maybe ground truth labels? but idk

    Returns:
        entities (List[dict[str, str])): ????
    """
    # label_map = dataset['medical_consultations'].features['ent_tags'].feature.names
    entities = []
    current_entity = []
    for token, tag in zip(tokens, ner_tags):
        # tag_label = label_map[tag]
        if tag_label.startswith("B-"):
            if current_entity:
                entities.append(" ".join(current_entity))
            current_entity = [token]
        elif tag_label.startswith("I-") and current_entity:
            current_entity.append(token)
        else:
            if current_entity:
                entities.append(" ".join(current_entity))
                current_entity = []
    if current_entity:
        entities.append(" ".join(current_entity))
    return entities

# results3 = pipe(truncated_text)

In [41]:
get_true_entities(lst_of_ent_tags, results)

AttributeError: 'Value' object has no attribute 'names'

In [1]:
lst_of_lst_of_dict = copy.deepcopy(lst_of_ent_tags)
tn,tp = 0,0
fn,fp = 0,0
for i, lst in tqdm(enumerate(results)):
    for dictionary in lst:
        start_idx = dictionary['start']
        end_idx = dictionary ['end']
        doc_i = lst_of_docs[i]
        tok = doc_i[start_idx:end_idx+1]
        ent_tags = lst_of_ent_tags[i]
        entity = dictionary['entity']
        doc_tok = lst_of_tokens[i]
        try:
            
            tok_idx = doc_tok.index(tok.strip())
        except ValueError as ve:
            fp += 1
            continue
# print(start_idx)
# print(end_idx)
# print(doc_i)
# print(tok)
# print(ent_tags)
# print(entity)
# print(doc_tok)
# raise ValueError(f'{start_idx} oh no') from ve

        try:
            
            tag = ent_tags[tok_idx]
        except IndexError as ve:
            print(doc_tok)
            print(ent_tags)
            assert len(doc_tok) == len(ent_tags), f'{len(doc_tok)} != {len(ent_tags)}'
        is_pii = tag != 'O'
        if is_pii: 
            tp += 0 
        else:
            
            fp += 1
        # print(tag)
        # lst_of_lst_of_dict = 
        lst_of_lst_of_dict[i].remove(tag)
    for tag in lst_of_lst_of_dict: 
        if tag == 'O':
            
            tnt = 1
        else:
             
             fnt = 1

NameError: name 'copy' is not defined

In [ ]:
lst_of_lst_of_dict_entities = []